In [ ]:
import os
import zipfile
import gdown
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# ================= CONFIGURATION =================
DATA_FILE_ID = '14djOtTxtCsFTsXlKKSK-zw37H1nIHyz9'
ZIP_NAME = "Real_Dataset_New.zip"
EXTRACT_DIR = "Real_Dataset_Final"
MODEL_FILE_ID = '1s05mTnjdlMRZevbN8xlBxeBHId-KgGZm' # HardMiner Model
MODEL_NAME = "best_model_hardminer.keras"

CLASSES = ['Bridge', 'Clean', 'CMP', 'Crack', 'LER', 'Open', 'Other', 'Via']
MIN_SAMPLES = 30

# ================= 1. DOWNLOAD & EXTRACT =================
if not os.path.exists(MODEL_NAME): gdown.download(f'https://drive.google.com/uc?id={MODEL_FILE_ID}', MODEL_NAME, quiet=False)
if not os.path.exists(ZIP_NAME): gdown.download(f'https://drive.google.com/uc?id={DATA_FILE_ID}', ZIP_NAME, quiet=False)
if os.path.exists(EXTRACT_DIR): import shutil; shutil.rmtree(EXTRACT_DIR)

with zipfile.ZipFile(ZIP_NAME, 'r') as z: z.extractall(EXTRACT_DIR)

# ================= 2. LOAD & BALANCE DATA =================
print("🔍 Scanning & Balancing Data...")
all_images, all_labels = [], []

def find_folder(root, target):
    for r, dirs, f in os.walk(root):
        for d in dirs:
            if d.lower() == target.lower() or d.lower() == target.lower() + 's':
                return os.path.join(r, d)
    return None

for idx, cls in enumerate(CLASSES):
    path = find_folder(EXTRACT_DIR, cls)
    if not path: continue
    class_imgs = []
    files = [f for f in os.listdir(path) if f.lower().endswith(('.png','.jpg','.jpeg'))]
    for f in files:
        try:
            img = tf.keras.utils.load_img(os.path.join(path, f), color_mode='grayscale', target_size=(224, 224))
            arr = tf.keras.utils.img_to_array(img)
            class_imgs.append(arr)
        except: pass

    count = len(class_imgs)
    if count == 0: continue
    if count < MIN_SAMPLES:
        multiplier = int(np.ceil(MIN_SAMPLES / count))
        class_imgs = class_imgs * multiplier
    all_images.extend(class_imgs)
    all_labels.extend([idx] * len(class_imgs))

X_real = np.array(all_images)
y_real = np.array(all_labels)

# ================= 3. DEFINE CLASS WEIGHTS (The Fix for 'Clean' Bias) =================
# We tell the model: "Missing an Open is 5x worse than missing a Clean"
class_weights = {
    0: 3.0,  # Bridge (Priority)
    1: 0.5,  # Clean (Suppress it - don't be lazy!)
    2: 1.0,  # CMP
    3: 1.0,  # Crack
    4: 1.0,  # LER
    5: 5.0,  # Open (HIGHEST PRIORITY)
    6: 1.0,  # Other
    7: 1.0   # Via
}

# ================= 4. MODEL SETUP =================
print(f"♻️  Loading HardMiner Brain...")

# Note: We removed the 'SobelLayer' from custom_objects because HardMiner doesn't need it.
# If this line crashes, it means the .keras file technically has a layer named "SobelLayer" inside it.
# If that happens, just add the class definition back. But usually, HardMiner models are standard.
try:
    base_model = tf.keras.models.load_model(MODEL_NAME, custom_objects={'focal_loss_fixed': lambda y, p: 0})
except:
    print("⚠️ Needed Sobel Class definition after all. Re-adding...")
    @tf.keras.utils.register_keras_serializable()
    class SobelLayer(layers.Layer):
        def __init__(self, **kwargs):
            super(SobelLayer, self).__init__(**kwargs)
            self.k = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=tf.float32)
            self.k = tf.reshape(self.k, [3, 3, 1, 1])
        def call(self, inputs): return inputs # Dummy pass-through if needed
        def get_config(self): return super(SobelLayer, self).get_config()
    base_model = tf.keras.models.load_model(MODEL_NAME, custom_objects={'SobelLayer': SobelLayer, 'focal_loss_fixed': lambda y, p: 0})


base_model.trainable = False
x = base_model.layers[-2].output
x = layers.Dense(64, activation='relu', name='dense_weighted_translator')(x)
x = layers.Dropout(0.2, name='dropout_weighted')(x)
new_output = layers.Dense(len(CLASSES), activation='softmax', name='dense_final_8class_weighted')(x)

model = models.Model(inputs=base_model.input, outputs=new_output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

datagen = ImageDataGenerator(
    rotation_range=180,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)
train_generator = datagen.flow(X_real, y_real, batch_size=8)

# ================= 5. TRAINING WITH WEIGHTS =================
print("🚀 Starting Weighted Fine-Tuning (30 Epochs)...")
model.fit(
    train_generator,
    steps_per_epoch=max(1, len(X_real)//8),
    epochs=30,
    verbose=1,
    class_weight=class_weights
)

# ================= 6. EXPORT =================
print("💾 Saving Weighted TFLite...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = lambda: ([tf.cast(img[None], tf.float32)] for img in X_real)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.allow_custom_ops = True

name = "MobileNetV2_8Class_Weighted.tflite"
with open(name, 'wb') as f: f.write(converter.convert())
print(f"✅ DONE! Download '{name}'")